# Task C_2: Streaming Application
## Please check the terminal to see the running process

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'
import sys
import time
import datetime
import json
import pandas as pd
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pprint import pprint
import geohash
#print('geohash for 88,22',geohash.encode(88,22,precision=5))

In [2]:
def joinDataAndSendToDB(iter):
    # create connection to Mongo and setup collection
    client = MongoClient()
    db = client.fit5148_assignment_db
    coll_hotspot = db.hotspot
    coll_climate = db.climate
    empty_list = []
    client = MongoClient()
    print('----------------')
    db = client.fit5148_assignment_db
    
    # set up 3 lists and append data from different producer accordingly
    list_producer1 = []
    list_producer2 = []
    list_producer3 = []
    list_hotspot_backup = []
    for record in iter:
        data = json.loads(record[1])
        sender_id = data['sender_id']
        print(sender_id)
        if sender_id == 'producer_1':
            print('climate')
            print(data)
            list_producer1.append(data)
        elif sender_id == 'producer_2':
            print('aqua')
            print(data)
            list_producer2.append(data)
        else: 
            print('terrar')
            print(data)
            list_producer3.append(data)       
    print('Start finding stuff to join')
    
    # if there are no record from both satellite, insert the climate data to Mongo directly
    if len(list_producer2) == 0 and len(list_producer3) == 0:
        print("No data from both satellite")        
        try:
            coll_climate.insert(data)
        except Exception as ex:
            print("Exception Occured. Message: {0}".format(str(ex)))
    # If there is record coming from only one satellite, store the climate data and hotspot data into each collections
    elif len(list_producer2) == 0 and len(list_producer3) != 0:
        print("Only TERRA data") 
        for c in list_producer1:
            climate = {
                'date':c['created_time'],
                'air_temperature_celcius':c['air_temperature_celcius'],
                'relative_humandity':c['relative_humidity'],
                'windspeed_knots':c['windspeed_knots'],
                'max_wind_speed':c['max_wind_speed'],
                'precipitation':c['precipitation '],
                'latitude':c['latitude'],
                'longitude':c['longitude']
            }
            coll_climate.insert_one(climate)
        # compare the location using geohash, if the location matched then join the data by appending the created time from 
        for p in list_producer3:
            position3 = geohash.encode(p['latitude'],p['longitude'],precision = 5)
            for c in list_producer1:
                position1 = geohash.encode(c['latitude'],c['longitude'],precision = 5)
                if position1 == position3:
                    hotspot = {
                    'latitude':p['latitude'],
                    'longitude':p['longitude'],
                    'confidence':p['confidence'],
                    'surface_temperature':p['surface_temperature'],
                    'created_time':c['created_time']    
                }
                    # insert the data to Mongo
                    coll_hotspot.insert_one(hotspot)                    
    elif len(list_producer2) != 0 and len(list_producer3) == 0:
        print("Only AQUA data")
        for c in list_producer1:
                    climate = {
                        'date':c['created_time'],
                        'air_temperature_celcius':c['air_temperature_celcius'],
                        'relative_humandity':c['relative_humidity'],
                        'windspeed_knots':c['windspeed_knots'],
                        'max_wind_speed':c['max_wind_speed'],
                        'precipitation':c['precipitation '],
                        'latitude':c['latitude'],
                        'longitude':c['longitude']
                    }
                    coll_climate.insert_one(climate)
            
        for p in list_producer2:
            position3 = geohash.encode(p['latitude'],p['longitude'],precision = 5)
            for c in list_producer1:
                position1 = geohash.encode(c['latitude'],c['longitude'],precision = 5)
                if position1 == position3:
                    hotspot = {
                    'latitude':p['latitude'],
                    'longitude':p['longitude'],
                    'confidence':p['confidence'],
                    'surface_temperature':p['surface_temperature'],
                    'created_time':c['created_time']    
                }
                    # insert the data to Mongo
                    coll_hotspot.insert_one(hotspot)
    # if there is data coming from both satellite, then compare the data and average the confidence and surface_temperature if the location matched                    
    elif len(list_producer2) != 0 and len(list_producer3) != 0:
        print("Receive data from both satellites")
        for c in list_producer1:
                    climate = {
                        'date':c['created_time'],
                        'air_temperature_celcius':c['air_temperature_celcius'],
                        'relative_humandity':c['relative_humidity'],
                        'windspeed_knots':c['windspeed_knots'],
                        'max_wind_speed':c['max_wind_speed'],
                        'precipitation':c['precipitation '],
                        'latitude':c['latitude'],
                        'longitude':c['longitude']
                    }
                    coll_climate.insert_one(climate)
                    
        for i in list_producer2:
            position2 = geohash.encode(i['latitude'],i['longitude'],precision = 5)
            coll_hotspot
            for j in list_producer3:
                position3 = geohash.encode(j['latitude'],j['longitude'],precision = 5)
                if position2 == position3:
                    hotspot = {
                        'latitude':i['latitude'],
                        'longitude':i['longitude'],
                        'confidence':(i['confidence']+j['confidence'])/2,
                        'surface_temperature':(i['surface_temperature']+j['surface_temperature'])/2
                    } 
                    # Append all data to this list after average
                    list_hotspot_backup.append(hotspot)
        
        for h in list_hotspot_backup:
            position3 = geohash.encode(h['latitude'],h['longitude'],precision = 5)
            for c in list_producer1:
                position1 = geohash.encode(c['latitude'],c['longitude'],precision = 5)
                if position1 == position3:
                    hotspot = {
                    'latitude':h['latitude'],
                    'longitude':h['longitude'],
                    'confidence':h['confidence'],
                    'surface_temperature':h['surface_temperature'],
                    'created_time':h['created_time']    
                }
                    # insert joined hotspot into Mongo
                    coll_hotspot.insert_one(hotspot)
                          
    client.close()

# set the spark interval to 10 seconds
n_secs = 10

# set up topic to subscribe
topic = "StopFire"

# create connection
conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week11-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary
data = []
#print('success')
# perform join operation and send data to Mongo
lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(joinDataAndSendToDB))


ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)



KeyboardInterrupt: 